## Data Collection - Players

---

In [1]:
import pandas as pd
from functools import reduce

from client.models.player import Player
from client.scrapers.fifa import get_player_by_name

from client.scrapers.whoscored.player import crawl_player_overall_stats, crawl_player_defensive_stats, \
crawl_player_passing_stats, crawl_player_offensive_stats

In [2]:
league_team_df = pd.read_csv('./league/Premier League.csv')

In [3]:
players = []
for idx, row in league_team_df.iterrows():
    if idx < 1:
        # excute player data crawling functions 
        overall_df = crawl_player_overall_stats(int(row.team_id))
        defensive_df = crawl_player_defensive_stats(int(row.team_id))
        passing_df = crawl_player_passing_stats(int(row.team_id))
        offensive_df = crawl_player_offensive_stats(int(row.team_id))
    
        # merge dataframes
        players_df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'), [overall_df, defensive_df, passing_df, offensive_df])

        for i, player_row in players_df.iterrows():
            player: Player = get_player_by_name(player_name=player_row['name'], team_name=row.team_name)
            stats = player_row.to_dict()
            stats.pop('name')
            player.set_stats(data=player_row.to_dict())
            players.append(player)

In [4]:
['{} - g: {} a: {} d: {} w: {} v: {}'.format(
    p.get_stat('full_name'),
    p.get_stat('goals'),
    p.get_stat('assists'),
    p.get_stat('date_of_birth'),
    p.get_stat('wage'),
    p.get_stat('transfer_value')
) for p in players]

['Bukayo Saka - g: 4 a: 4 d: 2001-09-05 w: €150K v: €99M',
 'Declan Rice - g: 2 a: 1 d: 1999-01-14 w: €130K v: €63M',
 'Gabriel Teodoro Martinelli Silva - g: 1 a: 2 d: 2001-06-18 w: €145K v: €61.5M',
 'Martin Ødegaard - g: 3 a: 1 d: 1998-12-17 w: €170K v: €109M',
 'Thomas Partey - g: 0 a: 0 d: 1993-06-13 w: €150K v: €45M',
 'Edward Nketiah - g: 5 a: 0 d: 1999-05-30 w: €74K v: €8.5M',
 'Gabriel Fernando de Jesus - g: 1 a: 0 d: 1997-04-03 w: €165K v: €50.5M',
 'Benjamin White - g: 1 a: 0 d: 1997-10-08 w: €95K v: €25.5M',
 'Oleksandr Zinchenko - g: 1 a: 0 d: 1996-12-15 w: €100K v: €24M',
 'Leandro Trossard - g: 3 a: 1 d: 1994-12-04 w: €130K v: €26.5M',
 'Gabriel dos Santos Magalhães - g: 0 a: 0 d: 1997-12-19 w: €130K v: €47M',
 'William Alain André Gabriel Saliba - g: 1 a: 1 d: 2001-03-24 w: €105K v: €53M',
 'David Raya Martin - g: 0 a: 0 d: 1995-09-15 w: €48K v: €19.5M',
 'Jurriën David Norman Timber - g: 0 a: 0 d: 2001-06-17 w: €80K v: €33M',
 'Kai Lukas Havertz - g: 1 a: 1 d: 1999-06-1